In [2]:
import numpy as np
import math
import itertools
import pprint

# Generation of Rotation Matrix from Orientation Relationship

Orientation relationships (ORs) are typically given in the form of one set of crystallographic direction vectors and one set of crystallographic planes. A second set of direction vectors may be given, however, these second direction vectors may always be recovered by find the direction vectors normal to the other two vector sets. 

To convert the ORs to rotation matricies, must convert each crystallographic orientation (as defined by the OR) to an orientation matrix. Therefore, for each phase in the OR, we have the vectors:

$$
\begin{align}
\vec{n} = \frac{(h,k,l)}{|hkl|}
\end{align}
$$

$$
\begin{align}
\vec{b} = \frac{(u,v,w)}{|uvw|}
\end{align}
$$

$$
\begin{align}
\vec{t} = \frac{\hat{n}\times\hat{b}}{|\hat{n}\times\hat{b}|}
\end{align}
$$

And the orientation matrix:

$$
\begin{align}
g_{i} =
\begin{bmatrix}
b_1 & t_1 & n_1 \\
b_2 & t_2 & n_2 \\
b_3 & t_3 & n_3
\end{bmatrix}
\end{align}
$$

The final rotation matrix between the two phases is calculated as:

$$
\begin{align}
g = g_c g_p^{-1}
\end{align}
$$

where $c,p$ denote child and parent phases

In [3]:
def or2rotm(orc1, orc3):
    # Finding the orientation matrix of the parent phase
    n_p = orc3[0,:]/np.linalg.norm(orc3[0,:]) # Must be the planar constraint
    b_p = orc1[0,:]/np.linalg.norm(orc1[0,:]) # Must be a directional constraint
    t_p = np.cross(n_p, b_p)/np.linalg.norm(np.cross(n_p, b_p))

    g_p = np.array([[b_p[0], t_p[0], n_p[0]],[b_p[1], t_p[1], n_p[1]],[b_p[2], t_p[2], n_p[2]]])


    # Finding the orientaiton matrix of the child phase
    n_c = orc3[1,:]/np.linalg.norm(orc3[1,:]) # Must be the planar constraint
    b_c = orc1[1,:]/np.linalg.norm(orc1[1,:]) # Must be a directional constraint
    t_c = np.cross(n_c, b_c)/np.linalg.norm(np.cross(n_c, b_c))

    g_c = np.array([[b_c[0], t_c[0], n_c[0]],[b_c[1], t_c[1], n_c[1]],[b_c[2], t_c[2], n_c[2]]])


    # Calculating the misorientation matrix between the two phases
    g = np.dot(g_c, g_p.T)
    
    return g

Furthermore, we can convert this rotation matrix into a set of Euler angles. Note that this capability is included in some MATLAB packages (which cost more money), however, they do not include the capability of converting to Euler angles using the ZX'Z' convention which is used by MOOSE. The equations necessary to do this conversion in the ZX'Z' convention can be found at https://en.wikipedia.org/wiki/Euler_angles#Rotation_matrix:

$$
\begin{align}
\phi_1 = \arctan{\bigg( \frac{g_{13}}{-g_{23}} \bigg)}
\end{align}
$$

$$
\begin{align}
\Psi = \arctan{\bigg( \frac{\sqrt{1-g_{33}^2}}{g_{33}} \bigg)}
\end{align}
$$

$$
\begin{align}
\phi_2 = \arctan{\bigg( \frac{g_{31}}{g_{32}} \bigg)}
\end{align}
$$

In [4]:
def rotm2eul(notation, rotm):
    if notation == 'ZXZ':
        phi1 = math.atan(rotm[0,2]/-rotm[1,2])
        PHI  = math.atan((math.sqrt(1.0-(rotm[2,2]**2)))/(rotm[2,2]))
        phi2 = math.atan(rotm[2,0]/rotm[2,1])
        eulerAngles = np.degrees(np.array([phi1, PHI, phi2]))
        return eulerAngles
    else:
        raise RuntimeError('Not a valid Euler angle notation')

# Ferrite & Martensite in Austenite

## Nishiyama-Wassermann Variants

The Nishiyama-Wassermann ORs are a set of 12 ORs which had been observed in the fcc-austenite to bcc-ferrite (or bct phase caused by tetragonalization due to high carbon content) phase transformation. 

In [7]:
#                      directional                planar
#                     AUS       FER           AUS       FER

NWors = np.array([[[[1,0,-1], [1,0,0]],    [[1,1,1], [0,1,1]]], #NW01
                  [[[-1,1,0], [1,0,0]],    [[1,1,1], [0,1,1]]], #NW02
                  [[[0,-1,1], [1,0,0]],    [[1,1,1], [0,1,1]]], #NW03
                  [[[1,0,1],  [1,0,0]],    [[-1,1,1],[0,1,1]]], #NW04
                  [[[-1,-1,0],[1,0,0]],    [[-1,1,1],[0,1,1]]], #NW05
                  [[[0,1,-1], [1,0,0]],    [[-1,1,1],[0,1,1]]], #NW06
                  [[[-1,0,1], [1,0,0]],    [[1,-1,1],[0,1,1]]], #NW07
                  [[[1,1,0],  [1,0,0]],    [[1,-1,1],[0,1,1]]], #NW08
                  [[[0,-1,-1],[1,0,0]],    [[1,-1,1],[0,1,1]]], #NW09
                  [[[-1,0,-1],[1,0,0]],    [[1,1,-1],[0,1,1]]], #NW10
                  [[[1,-1,0], [1,0,0]],    [[1,1,-1],[0,1,1]]], #NW11
                  [[[0,1,1],  [1,0,0]],    [[1,1,-1],[0,1,1]]], #NW12
                 ])

In [8]:
printnum = 3

for i in range(0,printnum):
    rotm = or2rotm(NWors[i,0], NWors[i,1])
    eul = rotm2eul('ZXZ', rotm)
    print(eul)

[ 80.4019697   45.81931182 -76.36129272]
[ -0.         -80.26438968  45.        ]
[-80.4019697   45.81931182 -13.63870728]


## Kurdjumov-Sachs Variants

The Kurdjumov-Sachs (KS) varients are a series of ORs published in response to the Nishiyama-Wassermann ORs. It contains a set of 24 ORs which had been observed in the fcc-austenite to bcc-ferrite (or bct phase caused by tetragonalization due to high carbon content) phase transformation. 

In [5]:
#                      directional                planar
#                     AUS       FER           AUS       FER

KSors = np.array([[[[1,0,-1], [1,1,-1]],    [[1,1,1], [0,1,1]]], #KS01
                  [[[1,0,-1], [-1,1,-1]],   [[1,1,1], [0,1,1]]], #KS02
                  [[[-1,1,0], [1,1,-1]],    [[1,1,1], [0,1,1]]], #KS03
                  [[[-1,1,0], [-1,1,-1]],   [[1,1,1], [0,1,1]]], #KS04
                  [[[0,-1,1], [1,1,-1]],    [[1,1,1], [0,1,1]]], #KS05
                  [[[0,-1,1], [-1,1,-1]],   [[1,1,1], [0,1,1]]], #KS06
                  [[[1,0,1],  [1,1,-1]],    [[-1,1,1],[0,1,1]]], #KS07
                  [[[1,0,1],  [-1,1,-1]],   [[-1,1,1],[0,1,1]]], #KS08
                  [[[-1,-1,0],[1,1,-1]],    [[-1,1,1],[0,1,1]]], #KS09
                  [[[-1,-1,0],[-1,1,-1]],   [[-1,1,1],[0,1,1]]], #KS10
                  [[[0,1,-1], [1,1,-1]],    [[-1,1,1],[0,1,1]]], #KS11
                  [[[0,1,-1], [-1,1,-1]],   [[-1,1,1],[0,1,1]]], #KS12
                  [[[-1,0,1], [1,1,-1]],    [[1,-1,1],[0,1,1]]], #KS13
                  [[[-1,0,1], [-1,1,-1]],   [[1,-1,1],[0,1,1]]], #KS14
                  [[[1,1,0],  [1,1,-1]],    [[1,-1,1],[0,1,1]]], #KS15
                  [[[1,1,0],  [-1,1,-1]],   [[1,-1,1],[0,1,1]]], #KS16
                  [[[0,-1,-1],[1,1,-1]],    [[1,-1,1],[0,1,1]]], #KS17
                  [[[0,-1,-1],[-1,1,-1]],   [[1,-1,1],[0,1,1]]], #KS18
                  [[[-1,0,-1],[1,1,-1]],    [[1,1,-1],[0,1,1]]], #KS19
                  [[[-1,0,-1],[-1,1,-1]],   [[1,1,-1],[0,1,1]]], #KS20
                  [[[1,-1,0], [1,1,-1]],    [[1,1,-1],[0,1,1]]], #KS21
                  [[[1,-1,0], [-1,1,-1]],   [[1,1,-1],[0,1,1]]], #KS22
                  [[[0,1,1],  [1,1,-1]],    [[1,1,-1],[0,1,1]]], #KS23
                  [[[0,1,1],  [-1,1,-1]],   [[1,1,-1],[0,1,1]]], #KS24
                 ])

In [6]:
printnum = 6

for i in range(0,printnum):
    rotm = or2rotm(KSors[i,0], KSors[i,1])
    eul = rotm2eul('ZXZ', rotm)
    print(eul)

[-24.20342834  10.52877937  65.79657166]
[-77.33353069  49.47122063 -12.66646931]
[41.95489158 85.70366404 80.37863911]
[ 83.58843092  42.13367951 -75.61499742]
[-48.77270548  80.40593177   4.35739688]
[  4.35739688 -80.40593177  48.77270548]


# Cementite in Austenite

## Thompson-Howell Orientation Relationship

The Thompson-Howell OR is a single OR which had been observed in the fcc-austenite to orthorhombic cementite intermetallic phase transformation. 

In [10]:
#                      directional                planar
#                     AUS       CEM           AUS       CEM

THor = np.array([[[-1,2,-1], [3,0,-1]],    [[1,1,1], [-1,0,3]]])

In [11]:
rotm = or2rotm(THor[0], THor[1])
eul = rotm2eul('ZXZ', rotm)
print(eul)

[38.86608588 47.40420053 66.8402483 ]


# Pearlite Homogenization

## The Bagaryatskii Orientation Relationship

The Bagaryatski OR is perhaps the most cited OR for cementite and ferrite in a lamellar pearlite structure. It is given by:

$$
\begin{align}
[100]_\theta \;||\; [1 \bar{1} 0]_\alpha
\end{align}
$$

$$
\begin{align}
[010]_\theta \;||\; [111]_\alpha
\end{align}
$$

$$
\begin{align}
(001)_\theta \;||\; (\bar{1} \bar{1} 2)_\alpha
\end{align}
$$

We will make the decision that the cementite phase is the parent phase, thus the rotation matrix will represent the rotation of the ferrite phase in relation to the cementite phase. Entering this into numpy arrays:

In [104]:
orc1 = np.array([[1, 0, 0],[1, -1, 0]])
orc2 = np.array([[0, 1, 0],[1, 1, 1]]) # Optional
orc3 = np.array([[0, 0, 1],[-1, -1, 2]]) # This is the planar OR condition

We can then perform the conversion of this orientation relationship into a rotation matrix, along with a set of Euler angles in the ZX'Z' convention by calling our two functions

In [78]:
rotm = or2rotm(orc1, orc3)

print('The rotation matrix for the Bagaryatskii OR is:\n' + str(rotm) + '\n')

eul = rotm2eul('ZXZ', rotm)

print('The Euler angles (in degrees) for the Bagaryatskii OR is:\n' + str(eul) + '\n')

The rotation matrix for the Bagaryatskii OR is:
[[ 0.70710678  0.57735027 -0.40824829]
 [-0.70710678  0.57735027 -0.40824829]
 [ 0.          0.57735027  0.81649658]]

The Euler angles (in degrees) for the Bagaryatskii OR is:
[-45.          35.26438968   0.        ]



For pearlite homogenization, we can imagine pearlite as a periodically layered composite consisting of ferrite and cementite lamellae. The first step is to use the rotation matrix of the ferrite phase which we just calculated using the Bagaryatskii OR to rotate the stiffness tensor. This is complicated unless we convert the 6x6 Voigt matrix into a 3x3x3x3 4th order tensor, apply the rotation, then convert back to the 6x6 Voigt matrix. We can then rotate the 4th order tensor using the equation

$$
\begin{align}
C_{mnop} = g_{mi}g_{nj}g_{ok}g_{pl} C_{ijkl}
\end{align}.
$$


In [113]:
Voigt_notation = [(0, 0), (1, 1), (2, 2), (1, 2), (0, 2), (0, 1)]

def full_3x3_to_Voigt_6_index(i, j):
    if i == j:
        return i
    return 6-i-j


def Voigt_6x6_to_full_3x3x3x3(C):
    C = np.asarray(C)
    C_out = np.zeros((3,3,3,3), dtype=float)
    for i, j, k, l in itertools.product(range(3), range(3), range(3), range(3)):
        Voigt_i = full_3x3_to_Voigt_6_index(i, j)
        Voigt_j = full_3x3_to_Voigt_6_index(k, l)
        C_out[i, j, k, l] = C[Voigt_i, Voigt_j]
    return C_out


def full_3x3x3x3_to_Voigt_6x6(C, tol=1e-3):
    C = np.asarray(C)
    Voigt = np.zeros((6,6))
    for i in range(6):
        for j in range(6):
            k, l = Voigt_notation[i]
            m, n = Voigt_notation[j]
            Voigt[i,j] = C[k,l,m,n]
    return Voigt


def rotate_elastic_constants(C, A, tol=1e-6):
    A = np.asarray(A)

    # Is this a rotation matrix?
    if np.sometrue(np.abs(np.dot(np.array(A), np.transpose(np.array(A))) -
                          np.eye(3, dtype=float)) > tol):
        raise RuntimeError('Matrix *A* does not describe a rotation.')

    # Rotate
    return full_3x3x3x3_to_Voigt_6x6(np.einsum('ia,jb,kc,ld,abcd->ijkl',
                                               A, A, A, A,
                                               Voigt_6x6_to_full_3x3x3x3(C)))

Using the ferrite and cementite matricies calculated in the Steel Elastic Properties document, we have the rotated ferrite matrix:

In [114]:
FER_0K = np.array([[247.675,143.326,143.326,      0,      0,      0],
                   [143.326,247.675,143.326,      0,      0,      0],
                   [143.326,143.326,247.675,      0,      0,      0],
                   [      0,      0,      0,119.705,      0,      0],
                   [      0,      0,      0,      0,119.705,      0],
                   [      0,      0,      0,      0,      0,119.705]])

CEM_0K = np.array([[380.000,170.625,164.625,      0,      0,      0],
                   [170.625,347.500,159.000,      0,      0,      0],
                   [164.625,159.000,336.875,      0,      0,      0],
                   [      0,      0,      0,20.8750,      0,      0],
                   [      0,      0,      0,      0,130.500,      0],
                   [      0,      0,      0,      0,      0,118.750]])

FER_0K_ROT = rotate_elastic_constants(FER_0K, g)

print(FER_0K_ROT.round(3))


[[330.212  90.802 113.312  -7.503  -7.503  15.007]
 [ 90.802 330.212 113.312  -7.503  -7.503  15.007]
 [113.312 113.312 307.702  15.007  15.007 -30.014]
 [ -7.503  -7.503  15.007  89.691 -30.014  -7.503]
 [ -7.503  -7.503  15.007 -30.014  89.691  -7.503]
 [ 15.007  15.007 -30.014  -7.503  -7.503  67.181]]


Under the assumption of Voigt, we can find the homogenized pearlite stiffness tensor by taking a weighted average of each of the tenor components. 

In [115]:
f_FER = 0.85
f_CEM = 1.0 - f_FER

homog_voigt = np.zeros((6,6))

for i in range(0,6):
    for j in range(0,6):
        homog_voigt[i,j] = f_FER*FER_0K_ROT[i,j] + f_CEM*CEM_0K[i,j]
        
print(homog_voigt.round(3))

[[337.68  102.776 121.009  -6.378  -6.378  12.756]
 [102.776 332.805 120.166  -6.378  -6.378  12.756]
 [121.009 120.166 312.078  12.756  12.756 -25.512]
 [ -6.378  -6.378  12.756  79.369 -25.512  -6.378]
 [ -6.378  -6.378  12.756 -25.512  95.813  -6.378]
 [ 12.756  12.756 -25.512  -6.378  -6.378  74.917]]


More complex mixing models can be found in the work of Dr. Wenbin Yu (https://www.sciencedirect.com/science/article/pii/S0020768306004264), as well as models which consider interfacial properties, which are almost certainly necessary for lamellar structures on the nano-scale (https://www.sciencedirect.com/science/article/pii/S0263822316309837?casa_token=wr-0plXUxPEAAAAA:2OCmn0No-AVtH-Yqey7SL0WpXDqxvy86CZNUuRk9RUdCUb5PuEbpGUWIoDqN-xKSPRy383nvcg)